# pd.get_dummies,OneHotEncoder (with small no of categories & many categories), Mean encoding.

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('train.csv')
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [3]:
df.nunique()

id                      381109
Gender                       2
Age                         66
Driving_License              2
Region_Code                 53
Previously_Insured           2
Vehicle_Age                  3
Vehicle_Damage               2
Annual_Premium           48838
Policy_Sales_Channel       155
Vintage                    290
Response                     2
dtype: int64

From above we can observe that Gender,Vehicle_Damage,Vehicle_Age can be OneHotEncoded as their unique value counts is less

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


We can use pd.get_dummies or OneHotEncoder for the same

In [5]:
pd.get_dummies(df).head()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Gender_Female,Gender_Male,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes
0,1,44,1,28.0,0,40454.0,26.0,217,1,0,1,0,0,1,0,1
1,2,76,1,3.0,0,33536.0,26.0,183,0,0,1,1,0,0,1,0
2,3,47,1,28.0,0,38294.0,26.0,27,1,0,1,0,0,1,0,1
3,4,21,1,11.0,1,28619.0,152.0,203,0,0,1,0,1,0,1,0
4,5,29,1,41.0,1,27496.0,152.0,39,0,1,0,0,1,0,1,0


In [6]:
from sklearn.preprocessing import OneHotEncoder
oneHot=OneHotEncoder(drop='first')
oneHot.fit(df[['Gender','Vehicle_Damage','Vehicle_Age']])
chk=oneHot.transform(df[['Gender','Vehicle_Damage','Vehicle_Age']]).toarray()
print(chk)

[[1. 1. 0. 1.]
 [1. 0. 0. 0.]
 [1. 1. 0. 1.]
 ...
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 0. 0. 0.]]


In [7]:
df.shape

(381109, 12)

In [8]:
df['Region_Code'].nunique()

53

In [9]:
df['Region_Code'].value_counts().sort_values(ascending=False).head(20).sum()

306321

In [10]:
306321/381109

0.803762178274457

If we OneHot Encode Region_code we will get 53 new columns and it might results in curse of dimensionality. To avoid this we can use the technique of selecting the frequent no of state codes based on requirement and consider the others as a single category .From above can see that out of all the rows, 80% of data has 20 region codes only and we can onehot encode them.And Region code can not ordinal/numerical type.

In [11]:
df['Region_Code']=df['Region_Code'].astype('object')
df['Region_Code'].dtypes

dtype('O')

In [12]:
top_20=[x for x in df['Region_Code'].value_counts().sort_values(ascending=False).head(20).index]
print(top_20)

[28.0, 8.0, 46.0, 41.0, 15.0, 30.0, 29.0, 50.0, 3.0, 11.0, 36.0, 33.0, 47.0, 35.0, 6.0, 45.0, 37.0, 18.0, 48.0, 14.0]


In [13]:
for val in top_20:
    col='Region_'+str(int(val))
    df[col]=np.where(df['Region_Code']==val,1,0)
df.head(10)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,...,Region_36,Region_33,Region_47,Region_35,Region_6,Region_45,Region_37,Region_18,Region_48,Region_14
0,1,Male,44,1,28,0,> 2 Years,Yes,40454.0,26.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Male,76,1,3,0,1-2 Year,No,33536.0,26.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Male,47,1,28,0,> 2 Years,Yes,38294.0,26.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Male,21,1,11,1,< 1 Year,No,28619.0,152.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Female,29,1,41,1,< 1 Year,No,27496.0,152.0,...,0,0,0,0,0,0,0,0,0,0
5,6,Female,24,1,33,0,< 1 Year,Yes,2630.0,160.0,...,0,1,0,0,0,0,0,0,0,0
6,7,Male,23,1,11,0,< 1 Year,Yes,23367.0,152.0,...,0,0,0,0,0,0,0,0,0,0
7,8,Female,56,1,28,0,1-2 Year,Yes,32031.0,26.0,...,0,0,0,0,0,0,0,0,0,0
8,9,Female,24,1,3,1,< 1 Year,No,27619.0,152.0,...,0,0,0,0,0,0,0,0,0,0
9,10,Female,32,1,6,1,< 1 Year,No,28771.0,152.0,...,0,0,0,0,1,0,0,0,0,0


Other technique that we can use for such categorical types is mean encoding by using the target variable

In [14]:
region_mean_dict=dict(df.groupby('Region_Code')['Response'].mean().round(3))
region_mean_dict

{0.0: 0.086,
 1.0: 0.108,
 2.0: 0.071,
 3.0: 0.128,
 4.0: 0.158,
 5.0: 0.116,
 6.0: 0.07,
 7.0: 0.132,
 8.0: 0.096,
 9.0: 0.082,
 10.0: 0.06,
 11.0: 0.113,
 12.0: 0.098,
 13.0: 0.09,
 14.0: 0.09,
 15.0: 0.072,
 16.0: 0.067,
 17.0: 0.07,
 18.0: 0.132,
 19.0: 0.163,
 20.0: 0.115,
 21.0: 0.071,
 22.0: 0.063,
 23.0: 0.153,
 24.0: 0.142,
 25.0: 0.043,
 26.0: 0.072,
 27.0: 0.074,
 28.0: 0.187,
 29.0: 0.123,
 30.0: 0.074,
 31.0: 0.101,
 32.0: 0.086,
 33.0: 0.099,
 34.0: 0.076,
 35.0: 0.125,
 36.0: 0.08,
 37.0: 0.079,
 38.0: 0.192,
 39.0: 0.124,
 40.0: 0.118,
 41.0: 0.122,
 42.0: 0.074,
 43.0: 0.092,
 44.0: 0.041,
 45.0: 0.11,
 46.0: 0.103,
 47.0: 0.088,
 48.0: 0.102,
 49.0: 0.075,
 50.0: 0.063,
 51.0: 0.153,
 52.0: 0.124}

In [15]:
list_vals=[]
for val in df['Region_Code']:
    if val in region_mean_dict.keys():
        list_vals.append(region_mean_dict[val])
    else:
        list_vals.append(0)
print(len(list_vals))

381109


In [16]:
df['Region_Code_Mean']=list_vals
df['Region_Code_Mean']

0         0.187
1         0.128
2         0.187
3         0.113
4         0.122
          ...  
381104    0.072
381105    0.079
381106    0.074
381107    0.090
381108    0.123
Name: Region_Code_Mean, Length: 381109, dtype: float64

OrdinalEncoder is used there is an order like low,medium,high etc..Here in this data set there is no ordianl value.but lets check how it works on column Vehicle_Age

In [17]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder().fit(np.array(df['Vehicle_Age']).reshape(-1,1))
chk=enc.transform(np.array(df['Vehicle_Age']).reshape(-1,1))
print(chk[0:10])

[[2.]
 [0.]
 [2.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]]


In [18]:
u_list=df['Vehicle_Age'].unique()
print(u_list)

['> 2 Years' '1-2 Year' '< 1 Year']
